<br>

**<font size="5">Table of contents</font>** 

 - [**Chapter 0: Introduction**](#Chapter0)
     - [**0.1 Statistical methods**](#0.1)
     - [**0.2 Data and portfolio construction**](#0.2)
 - [**Chapter 1: Data and portfolio construction**](#Chapter1)
    - [**1.1 High Minus Low (HML) portfolio**](#1.1)
    - [**1.2 Winners Minus Losers (WML) portfolio**](#1.2)
    - [**1.3 Value Momemtum (HML-WML) Portfolio**](#1.3) 
    - [**1.4 Market Portfolio (Mkt-RF) portfolio**](#1.4)
 - [**Chapter 2:  Risk- risk reward ratio's and Data exploration**](#Chapter2)
    - [**2.1 Return and risk reward ratio's**](#2.1)
    - [**2.2 Exploratory Data Analysis (EDA)**](#2.2)
 - [**Chapter 3: Simulation analysis Part 1**](#Chapter3)
    - [**3.1 Mkt-RF**](#3.1)
    - [**3.2 HML**](#3.2)
    - [**3.3 WML**](#3.3)
    - [**3.4 HML-WML**](#3.4)
    - [**3.5 Summarize Results**](#3.5)
 - [**Chapter 4:  Simulation analysis Part 2**](#Chapter4)
    - [**4.1.Simulation Period: 01-01-1963 till 31-12-2003**](#4.1)
        - [**4.1.1 Mkt-RF**](#4.1.1)
        - [**4.1.2 HML**](#4.1.2)
        - [**4.1.3 WML**](#4.1.3)
        - [**4.1.4 HML-WML**](#4.1.4)
    - [**4.2.Simulation Period: 01-01-1927 till 31-12-2003**](#4.2)
        - [**4.2.1 Mkt-RF**](#4.2.1)
        - [**4.2.2 HML**](#4.2.2)
        - [**4.2.3 WML**](#4.2.3)
        - [**4.2.4 HML-WML**](#4.2.4)
    - [**4.3 Summarize Results**](#4.3)
        - [**4.3.1 Simulation Period: 01-01-1963 till 31-12-2003**](#4.3.1)
        - [**4.3.2 Simulation Period: 01-01-1927 till 31-12-2003**](#4.3.2)
 - [**Chapter 5:  Conclusion and discussion**](#Chapter5)
 - [**Chapter 6:  Limitations**](#Chapter6)
 - [**Chapter 7: Questions**](#Chapter7)
 - [**Chapter 8: References**](#Chapter8)


<a id='Chapter0'></a>
# Chapter 0: Introduction

## Goal
 

The goal of this notebook is to twofold: 

 1. Try to have a better understanding about the return risk-profile of the **value (HML) factor, momentum (WML) factor and a combined portfolio of value and momentum (HML-WML)** . I also include the **market factor (Mkt-RF)** as a sort of benchmark. For now, I've focused on **geometric return (annualized) and maximum drawdown**. In case of **maximum drawdown**, I am concerned whether it make sense to bootstrap this statistic, since this an extremum  statistic. I have seen paper bootstrapping this statistic (e.g.[**[1]**](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3331680)).

2. Try to answer wether recent performance of value and momentum are in line with historical performance. Several people have noticed that most factors did not deliver much return in the last 15 years. Are the value and momentum broken or just experiencing a period of bad luck? 

## Statistical methods <a id='0.1'></a>
To answer these question I use different simulation assumptions about the behavior of factor returns:


 - **Simulation under normality** approximates factor returns by normal distributions
that I calibrate using the sample means and standard deviations from the full
sample.


- **The independent bootstrap** 
Resamples each factor’s return independently with replacement (i.e., allowing observations to
be chosen multiple times) from every other factor. This bootstrap scheme
preserves the empirical distributions of factor returns that is, it accounts for
deviations from normality, but does not account for serial correlations in factor returns.


- **Block bootstrap**: The independent bootstrap requires that the data being bootstrapped is independent and identically distributed (iid). This does not work well for time series, where serial correlation is present. One approach that addresses this limitation is block bootstrapping. There are several versions of the block bootstrap, I will look at the:  
<br> 
    - **Circular block bootstrap (CBB)**: This bootstrap method is an improvement on the **moving block bootstrap (MBB)**. The MBB randomly draws fixed size blocks from the data and cut and pastes them to form a new series the same size as the original data. However, it has a major limitation in that beginning and ending points are systematically underrepresented. To address this limitation an extension to this method was developed called the Circular Block Bootstrap (CBB). This approach is much the same except that it wraps around the beginning and ending points to ensure they are drawn with similar probability as the other blocks To address this limitation an extension to this method was developed called the Circular Block Bootstrap (CBB). This approach is much the same except that it wraps around the beginning and ending points to ensure they are drawn with similar probability as the other blocks. 
<br>

    - **Stationary bootstrap (SB)**: One limitation of the circular block bootstrap is the fixed block size. Different block sizes emphasize different periods or lengths of autocorrelation (memory). At the extremes you can take a block size so small that no serial correlation is captured, and at the other end you could take a block size so large that you end up sampling the original series. To address the fixed block sizes, the Stationary Bootstrap (SB) randomly samples blocks from a geometric distribution with mean $k$. The advantage of this approach is that the circular block bootstrap doesn't quite give us a stationary time series (The distribution of $R_{k-1:k}$ is not the same as the distribution of $R_{k:k+1}$). Averaging over the random choices of block lengths overcomes this problem.

I found an excellent implementation of these methods from the [**ARCH package [13]**](https://arch.readthedocs.io/en/latest/bootstrap/timeseries-bootstraps.html). 

**Bootstrap implementation**

When bootstrapping the simulated paths for different investment horizons, I use the total length of the observed path to sample from. As a result, I obtain a  matrix where each column is a simulated path. Similar as **[Fama and French [2]](https://academic.oup.com/raps/article-abstract/8/2/232/4810768)** I then calibrate the number of rows of this matrix depending on the investment horizon.


## Data and portfolio construction <a id='0.2'></a>

I will be using daily returns (possible to change to monthy) obtained from [**Kenneth French's data library [8]**](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) for the US stock market (NYSE, AMEX, and NASDAQ) over the period  03/01/1927 to 28/02/2019. The data can be sourced directly from their website using [**Pandas datareader [14]**](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html). Note thate the Fama-French returns are simple raw returns (not log returns).

**As a reminder:**

 - A **value strategy** favors high fundamentals-to-price ratios by going long these stocks, while selling short those stocks
that have lower fundamentals-to-price ratios.
 - On the other hand, a **momentum strategy** exploits the phenomenon that securities which have performed well relative
to peers (winners) on average continue to outperform, and securities that have performed relatively poorly (losers) tend to
continue to under-perform.

**Long-Short Portfolio Construction:**

There are different ways to construct long-short value and momentum portfolio's. For now I will construct the portfolio's as Fama and French did in their 2012 [**paper [3]**](https://www.sciencedirect.com/science/article/pii/S0304405X12000931):

  -  **Value Portfolio (HML):** is formed by first splitting the universe of stocks into two size categories: `Small (S)  and Big (B)` using NYSE market cap medians and then splitting stocks `into three groups based on book-to-market equity highest 30% (H), middle 40% (M), and lowest 30% (L)`, using NYSE breakpoints. The intersection of stocks across the six categories are value-weighed and used to form the portfolios SH (small, high $\frac{BE}{ME}$), SM (small, middle $\frac{BE}{ME}$), SL (small, low $\frac{BE}{ME}$), BH (big, high $\frac{BE}{ME}$), BM (big, middle $\frac{BE}{ME}$), and BL (big, low $\frac{BE}{ME}$). HML is the average of the two high book-to-market portfolios minus the average of the two low book-to-market portfolios. 

\begin{equation} \text{HML} =	\frac{1}{2}\times  (\text{Small High Value} + \text{Big High Value})
 - \frac{1}{2}\times (\text{Small Low Value} + \text{Big Low Value})\end{equation}

 - **Momentum portfolio (WML):** is constructed similarly to HML in which six value-weight portfolios formed on size and prior (2-12) returns to construct momentum. Specifically, prior (2-12) returns is defined as the past 12-month return, skipping the most recent months return to avoid micro-structure and liquidity biases and now generally used as the standard definition of momentum. The portfolios, which are formed daily, are the `intersections of 2 portfolios formed on size (market equity, ME) and 3 portfolios formed on prior (2-12) return`. The daily size breakpoint is the median NYSE market equity. The daily prior (2-12) return breakpoints are the 30th and 70th NYSE percentiles. Momentum is the average return on the two high prior return portfolios (Winners) minus the average return on the two low prior return portfolios (Losers)

\begin{equation} \text{WML} =	\frac{1}{2}\times  (\text{Small Winners} + \text{Big Winners})
 - \frac{1}{2}\times (\text{Small Losers} + \text{Big Losers})\end{equation}


 - **Value-Momentum portfolio (HML-WML)**: is a `50-50 weighting` of the value and momentum strategy.
 

 -  **Market portfolio (Mkt-RF):** represents the equity market risk premium, or aggregate equity return minus the risk free
rate. It is the return from simply being long equities at market capitalization weights and, unlike other factors, is not a
spread return between

Other common implementations of value and momentum are based on deciles and quintiles. For example, Moskowitz and Daniel [[4](https://www.sciencedirect.com/science/article/pii/S0304405X16301490)] construct their momentum portfolios based on deciles where the winner portfolio is decile 1 and the loser portfolio decile 10. Formally, this can be represented as follow:

\begin{equation} \text{WML (Dec.)} =	\frac{1}{2}\times\text{Winners (D1)}
 - \frac{1}{2}\times \text{Losers (D10)}\end{equation}

If I have time left, I will take a closer look at the influence of portfolio construction based on `deciles and quintiles` (I did had very quick look  at the implementation based on deciles, when verifying results of Kent and Moskowitz, and noticed that  the implementation based on `deciles` earns a substantial higher premium, but also much higher drawdowns)

What follows is some of my initial experimentation with these simulation approaches along with some caveats. The analysis is organized as follow: I start with construction the portfolios in [**Chapter 1**](#Chapter1). Next, I introduce some common performance statistics and perform some exploratory data analysis in [**Chapter 2**](#Chatper2). In [**Chapter 3**](#Chapter3), I start with part 1 of the simulation analysis where I try to gain a better understanding of the risk-return profile for the portfolios under different investment horizons. Subsequently, in [**Chapter 4**](#Chapter4), I look at the recent past performance of these portfolios and examine whether this is in line with historical performance. Finally, in [**Chapter 5**](#Chapter5) and [**Chapter 6**](#Chapter6) , I summarize my conclusions and discusses limitations of the study. 

**I have stated my questions regarding the analysis in** [**Chapter 7**](#Chapter7). 


At the top of each chapter I have declared which functions are being used. I have decided to put these functions into a separate file named `simulation_lib` to keep this notebook as well-organized as possible. Furthermore, note that I use strategies, portfolios, factors interchangeably throughout the analysis. They all have the same interpretation. 

**<font size="5">Import libraries</font>**

In [ ]:
from pathlib import Path

import datetime
import os 
import warnings
warnings.filterwarnings("ignore")
#os.chdir("C:\\Users\Pieter-Jan\Documents\Factor_Crashes\Poster")

# scientific libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats.mstats import gmean
import statsmodels.api as sm
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from random import randint
#import numba as nb
#from numba import jit
#from numba import jitclass


# plot libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import seaborn as sns
#import brewer2mpl
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib.ticker import FormatStrFormatter

# imports file with all custum functions
from importlib import reload 
import simulation_lib
reload(simulation_lib)
# instead of listing all individual functions, we chose here to load everything
# Make sure to use unique function names, though, or you might overrule built-in functions
from simulation_lib import *

# plot parameters
sns_params = {
    'font.family':'serif',
    'font.size': 12,
    'font.weight': 'medium',
    'figure.figsize': (10, 7),
}
plt.style.use('seaborn-talk')
plt.style.use('bmh')
sns.set_context(sns_params)
savefig_kwds = dict(dpi=300, bbox_inches='tight', frameon=True, format='png')
set2=['#348ABD', '#A60628', '#7A68A6', '#467821', '#D55E00', '#CC79A7', '#56B4E9']
#plt.style.available
#sns.set(style='darkgrid', context='talk')
%matplotlib inline

<a id='Chapter1'></a>
# Chapter 1: Data and portfolio construction

I start with constructing the four portfolios:
 - The market portfolio (Mkt-RF)
 - The value portfolio (HML)
 - The momentum portfolio (WML)
 - A 50-50 weighted value-momentum portfolio (HML-WML)
 
I will implement the long-short portfolio myself using the raw portfolio sorts to walk through implementation process. Alternatively, you could directly take the long short portfolio constructed by Fama and French. To verify whether my portfolio implementation is correct, I compare my portfolio construction with Fama and French's implementation. Ideally, I should get the exact same results. This is only applicable for the HML and WML portfolio. For the market portfolio there is no sorting involved since this is just a value weighted portfolio of all stocks ((NYSE, AMEX, and NASDAQ)) minus the risk free. Finally, for the HML-WML portfolio, I choose arbitrarily a 50-50 weighting.

### Helper function
 - `cumsum_plot:` plots the cumulative return 

### Global parameters
Global parameters are defined in CAPITAL LETTERS. This makes it is easy to quickly change from daily data to monthly and some other parameters.

In [ ]:
# data frequency, easy to change between month and daily data
FREQUENCY = 12
N_PATHS = 10_000  # nr ob paths (bootstraps)
BLOCKSIZE = 18  # blocksize, for the Stationary bootstrap this is the mean of the geometric distribution
IH = [1, 3, 5, 10, 20, 30]  # investment horizon in years
START_YEAR = 1927  # start year when sourcing the portfolios from FAMA AND FRENCH
PLOT_INTERMEDIATE = False  # plots intermediate histograms when calling the simulation methods

# Portfolio names from FAMA AND FRENCH ; to switch to monthly just remove '_daily'
VALUE = '6_Portfolios_2x3'
CHECK_VALUE = 'F-F_Research_Data_Factors'
MOMENTUM = '6_Portfolios_ME_Prior_12_2'
CHECK_MOMENTUM = 'F-F_Momentum_Factor'

In [ ]:
# get a look at all the available portfolios
print(len(get_available_datasets()))
get_available_datasets()[:5]

## High Minus Low (HML) portfolio <a id='1.1'></a>

In [ ]:
# load the data for the value factor
dic_HML = web.DataReader(VALUE, 'famafrench', start=START_YEAR)
print(dic_HML['DESCR'])

In [ ]:
df_HML = dic_HML[0]
df_HML.index = df_HML.index.astype('datetime64[ns]')
print(df_HML.head(n=4))
cumsum_plot(
    df_HML,
    title="Six Portfolio sorted on book-to-market (BM) and market equity (ME)")

### Construct HLM long short portfolio

In [ ]:
# long short value portfolio
longs, shorts = ['SMALL HiBM', 'BIG HiBM'], ['SMALL LoBM', 'BIG LoBM']
HML_LS = pd.DataFrame(df_HML.loc[:, longs].sum(axis=1) / 2 -
                      df_HML.loc[:, shorts].sum(axis=1) / 2,
                      columns=['HML_VW'])  # value weighted HML

In [ ]:
cumsum_plot(HML_LS, title="Long-short value portfolio (HML)")
HML_LS.head(n=5)

### Compare both HML long short portfolios
If my construction method is correct, we should get exact same results.

In [ ]:
dict_HML_check = web.DataReader(CHECK_VALUE, 'famafrench', start=START_YEAR)

In [ ]:
print(dict_HML_check['DESCR'])
HML_check = dict_HML_check[0]
HML_check.index = HML_check.index.astype('datetime64[ns]')
HML_check.head(n=5)

In [ ]:
Compare_HML = pd.DataFrame(HML_check.loc[:, "HML"].values,
                           columns=["HML_1"],
                           index=HML_check.index)
Compare_HML["HML_2"] = HML_LS.loc[:, "HML_VW"].values
cumsum_plot(Compare_HML,
            title="Comparison of the value portfolio implementation")

Seems identical to me. Now let's do the same for the momentum factor (WML).

## Winners Minus Losers (WML) portfolio <a id='1.2'></a>

In [ ]:
# load the data for the value factor
dic_WML = web.DataReader(MOMENTUM, 'famafrench', start=START_YEAR)
print(dic_WML['DESCR'])

In [ ]:
df_WML = dic_WML[0]
df_WML.index = df_WML.index.astype('datetime64[ns]')
df_WML.head(n=5)

In [ ]:
cumsum_plot(
    df_WML,
    title="Six Portfolio sorted on prior returns (2-12) and market equity (ME)"
)

### Construct WML long short portfolio

In [ ]:
# long short momentum portfolio
longs, shorts = ['SMALL HiPRIOR',
                 'BIG HiPRIOR'], ['SMALL LoPRIOR', 'BIG LoPRIOR']
WML_LS = pd.DataFrame(df_WML.loc[:, longs].sum(axis=1) / 2 -
                      df_WML.loc[:, shorts].sum(axis=1) / 2,
                      columns=['WML_VW'])  # momemtum weighted HML
# plot the cumulative return
cumsum_plot(WML_LS, title="Long-short momentum portfolio (WML)")
WML_LS.head(n=5)

In [ ]:
dict_WML_check = web.DataReader(CHECK_MOMENTUM, 'famafrench', start=START_YEAR)
print(dict_WML_check['DESCR'])
WML_check = dict_WML_check[0]
WML_check.index = WML_check.index.astype('datetime64[ns]')
WML_check.head(n=5)

### Compare both WML long short portfolios
 - if my construction method is correct this should be exactly the same

In [ ]:
Compare_WML = pd.DataFrame(WML_check.loc[:, "Mom   "].values,
                           columns=["WML_1"],
                           index=WML_check.index)

Compare_WML["WML_2"] = WML_LS.loc[:, "WML_VW"].values
cumsum_plot(Compare_WML,
            title="Comparison of the value portfolio implementation")

## 50-50 Value Momemtum Portfolio <a id='1.3'></a>

In [ ]:
w_HML, w_WML = 0.50, 0.50  #weights 50-50 weights (arbitrarely)

HML_WML = pd.DataFrame(HML_check.loc[:, ["HML"]].values * w_HML +
                       WML_check.loc[:, ["Mom   "]].values * w_WML,
                       columns=["HML-WML"],
                       index=WML_check.index)

cumsum_plot(HML_WML, title="Long-short value-momentum portfolio (HML-WML)")
HML_WML.head()

## Market Portfolio <a id='1.4'></a>

This is just the market (Mkt) portfolio minus the risk free rate (RF). I use this as the benchmark since basically every investor (including you and me) can buy this portfolio for almost no fee (there is no performance fee involved). You can think of this portfolio as the **[S&P500](https://en.wikipedia.org/wiki/S%26P_500_Index)**, but containing many more firms.

In [ ]:
cumsum_plot(pd.DataFrame(HML_check.loc[:, "Mkt-RF"], columns=["Mkt-RF"]),
            title="Market portfolio minus the risk free rate (Mkt-RF)")
HML_check.head()

## Compare 4 factors
   - Market factor (Mkt-RF)
   - Value factor (HML)
   - Momemtum factor (WML)
   - 50-50 value-momemtum portfolio (HML-WML)

In [ ]:
# append all four portfolios together in one dataframe
df_factors = pd.DataFrame(HML_check.loc[:, ["Mkt-RF", "HML"]])
df_factors["WML"] = WML_check.iloc[:, 0]
df_factors["HML-WML"] = HML_WML.iloc[:, 0]

# alternative legend labels
dic_fac = {
    "Mkt-RF": "Market minus risk free rate (Mkt-RF)",
    "HML": "Value (HML)",
    "WML": "Momentum (WML)",
    "HML-WML": "50-50 Momentum-Value (50-50 HML-WML)"
}

# plot
cumsum_plot(df_factors, dic=dic_fac, title="Portfolio comparison")
df_factors.head()

<a id='Chapter2'></a>
# Chapter 2:  Risk- risk reward ratio's and Data exploration

Before starting with the simulation exercise, I introduce some performance statistics and perform some data exploration. The goal is to gain insights that give us intuition about the data.

## Return and risk reward ratio's <a id='2.1'></a>

**In case when working with log returns:** 

- I was considering to change to log returns for computational speed, but then decided it was not worth the effort (maybe I will take another look)

The **geometric mean** of quantities $\left\{a_{1}, \dots, a_{n}\right\}$ is

\begin{equation}
\overline{a}_{g}=\left(\prod_{i=1}^{n} a_{i}\right)^{1 / n}
\end{equation}

Taking the logarithm of both sides gives
\begin{equation}
\log \overline{a}_{g}=\frac{1}{n} \sum_{i=1}^{n} \log a_{i}
\end{equation}


so the log of the geometric mean is equal to the arithmetic mean of the logs.
In  case, the relevant quantities $a_i$ are the growth rates over each period

\begin{equation}
a_{i}=1+r_{i}
\end{equation}

and plugging this into the above equation gives

\begin{equation}
\log \left(1+\overline{r}_{g}\right)=\frac{1}{n} \sum_{i=1}^{n} \log \left(1+r_{i}\right)
\end{equation}

\begin{equation}
\overline{r}_{g}=\exp \left(\frac{1}{n} \sum_{i=1}^{n} \log \left(1+r_{i}\right)\right)-1
\end{equation}


Sometimes the geomtric return is  approximated by the following formula:

\begin{equation}
\overline{r}_{g}=\frac{1}{n} \sum_{i=1}^{n} \log \left(1+r_{i}\right)
\end{equation}

this is because when the growth rate is small,

\begin{equation}
\log \left(1+\overline{r}_{g}\right) \approx \overline{r}_{g}
\end{equation}

it is often an approximation that you can get away with

**Simple raw returns**

- **Arithmetic return**: \begin{equation} R_{a} =
\frac{1}{n} \sum_{i=1}^{n} r_{i}=\frac{r_{1}+r_{2}+\cdots+r_{n}}{n}
\end{equation}

suppose our arithmetic return is at daily frequency, to annualize this I take $(R_a)\times 252$ (assuming 252 trading days in a year)

- **Geometric return**: \begin{equation} R_{g} = 
\sqrt[n]{\left(1+r_{1}\right) \times\left(1+r_{2}\right) \times \ldots\left(1+r_{n}\right)}
\end{equation}

suppose our geometric return is at daily frequency, to annualize this I take $(R_g)^{252} - 1$ (assuming 252 trading days in a year)


- **Sharpe ratio**: one of the most common risk-reward ratios used in finance is the Sharpe ratio. The Sharpe ratio discounts the expected excess returns of a portfolio $ {E(r)-r_{f}} $ by the volatility of the returns $\sigma_r$. Formally, this can be described as follow:


\begin{equation}
S=\frac{E\left(r_{e}\right)}{\sigma_{r}}=\frac{E(r)-r_{f}}{\sigma_{r}}
\end{equation}

where $\sigma=\sqrt{\sigma^{2}}=\sqrt{\frac{1}{N} \sum_{i=1}^{N}\left(r_{i}-\mu\right)^{2}} \text {and } \mu=\frac{1}{N} \sum_{i=1}^{N} r_{i}$

**Crude estimation of annualized volatility**

Most often volatility is annualized by multiplying by $\sqrt(n)$ where $n$ represents the time period (number of trading days or trading months). The formula uses the the fact that the standard deviation of the sum of $n$ independent variables (with equal standard deviations) is $\sqrt(n)$ times the standard deviation of the individual variables. This is only a crude approximation of the true volatility observed in financial markets and is only an accurate extrapolation if prices follow a random walk or Wiener process, whose steps have finite variance. As a result we will probably underestimate volatility. There are other ways (probably better) to measure volatility, but these are not the focus of this project.

**Lower Partial Moments (LPM)**

Whereas measures of risk-adjusted return based on volatility treat all deviations from the mean as risk, measures of risk-adjusted return based on lower partial moments  consider only deviations below some predefined minimum return threshold, $\tau$ as risk. For example, negative deviations from the mean is risky whereas positive deviations are not. A lower partial moment of order $j$ can be estimated from a sample of $k$ returns as follows: \begin{equation}
L P M_{j}(\tau)=\frac{1}{k} \sum_{i=1}^{k} \max \left(\tau-r_{i}, 0\right)^{j} \end{equation}


A useful classification of measures of risk-adjusted returns based on lower partial moments is by their order. The larger the order the greater the weighting will be on returns that fall below the target threshold, meaning that larger orders result in more risk-averse measures. Some popular measures using the equation above are:

   - **The Sortino ratio** was proposed as a modification to the Sharpe ratio. It discounts the excess return of a portfolio above a target threshold by the volatility of downside returns, $\delta^{2}$, instead of the volatility of all returns, $\sigma^2$. The volatility of downside returns is equivalent to the square-root second-order lower partial moment of returns. More formally this can be described as follow:
   
   \begin{equation}
S O R(\tau)=\frac{E\left(r_{e}\right)}{\delta^{2}}=\frac{E(r)-\tau}{\delta^{2}}=\frac{E(r)-\tau}{\sqrt{L P M_{2}(\tau)}}
\end{equation}
   

   - **The Omega ratio** discounts the excess returns of a portfolio above the target threshold, usually the risk-free rate, by the first-order lower partial moment of the returns. The first-order lower partial moment corresponds to the average expected loss also known as downside risk. Mathematically the omega ratio is defined as follow:
   
   \begin{equation}
\Omega(\tau)=\frac{E\left(r_{e}\right)}{L P M_{1(\tau)}}=\frac{E(r)-\tau}{L P M_{1}(\tau)}
\end{equation}
   

   - **The Kappa ratio** is a generalization of Omega and Sortino ratio first proposed in 2004 by Kaplan and Knowles.  It was shown that when the parameter $j$ of the Kappa ratio is set to one or two you get the Omega or Sortino ratio. The Kappa ratio is most often used with $j=3$ which is why it is often referred to as the Kappa 3 ratio. Mathematically this is defined as follow:
   
   \begin{equation}
K_{j}(\tau)=\frac{E\left(r_{e}\right)}{\sqrt[j]{L P M_{j}(\tau)}}=\frac{E(r)-\tau}{\sqrt[j]{L P M_{j}(\tau)}}
\end{equation}


Another measure to look at is risk is **Drawdown**.  It is the maximum decrease in the value of the portfolio over a specific period of time. Given the historical prices (values) for a portfolio, $S$, and a period of time, $t$, the drawdown of length $t$ over that period of time is the maximum distance between a previous two values $S_x$ and $S_{x−t}$,
   
   \begin{equation}
D(t)=\max \left\{0, \max _{t_{i} \in(0, t}\left\{S_{t_{i}}-S_{t_{i}-t}\right\}\right\}
\end{equation}


**Maximum drawdown** can be thought of as a list of drawdowns calculated from the same historical portfolio values, $S$, but for the whole time period (or for different sub time periods, rolling window). The maximum drawdown of a portfolio is the maximum decrease in portfolio value from a previous high to a new low.

**% Time underwater** is the percentage time the investment is in in a drawdown state. 

Personally, I like measures based on drawdowns since they are extremely intuitive and are IMO are a very natural way to look at risk. Contrary to other measures, drawdown is path dependent. Furthermore, I very much enjoyed the presentation of Rober Frey (Former Quant of the Medallion fund) on drawdown [**180 years of market drawdown [8]**](https://www.youtube.com/watch?v=27x632vOjXk&t=3349s).

### Helper functions
 - `Kappa_ratio:` calculates the kappa ratio 
 - `drawdown:` calculates drawdown
 - `Anonymous functions (lambda):` performance functions stored in a dictionary
 -  `perf_stats:` a wrapper around the anonymous functions to give the possibility for selecting the function(s) of interest
 - `Return_DD:`calculates the cumulative return and drawdown in one figure


In [ ]:
"""
some anonymous function to calculate various (performance) statistics,
these functions are stored in a dictionary

params:

    - df: pandas dataframe with the raw returns
    - time_freq: an integer indicating the number of trading days
    - treshold: a float determining the level of drawdown is taken into account
      when calculating the percentage of time an investment is in a drawdown state

returns:

    - numpy array with the calculted statistic
"""

P_AR = lambda df, time_freq=FREQUENCY: np.mean(df) * time_freq
P_GR = lambda df, time_freq=FREQUENCY: (gmean((1 + df / 100))**time_freq - 1
                                        ) * 100
P_AVol = lambda df, time_freq=FREQUENCY: df.std() * np.sqrt(time_freq)
P_GVol = lambda df, time_freq=FREQUENCY: (np.exp(np.std(np.log(1 + df / 100)))
                                          - 1) * np.sqrt(time_freq) * 100

# put functions in a dictionary, so we can choose which one to call and
# which not (for the simulation)
perf_stats_dic = {
    "Skew": (lambda df: df.skew().values),
    "Kurtosis": (lambda df: df.kurtosis().values),
    "Terminal Wealth Relative":
    (lambda df: (1 + df / 100).cumprod().iloc[-1, :] - 1),
    "Arithmetric Return (%)":
    (lambda df, time_freq=FREQUENCY: P_AR(df, time_freq).values),
    "Geometric Return (%)":
    (lambda df, time_freq=FREQUENCY: P_GR(df, time_freq)),
    "Arithmetric Volatility (%)":
    (lambda df, time_freq=FREQUENCY: P_AVol(df, time_freq).values),
    "Geometric Volatility (%)":
    (lambda df, time_freq=FREQUENCY: P_GVol(df, time_freq)),
    "Arithmetric SR": (lambda df, time_freq=FREQUENCY: P_AR(df, time_freq) /
                       P_AVol(df, time_freq)),
    "Geometric SR": (lambda df, time_freq=FREQUENCY: P_GR(df, time_freq) /
                     P_GVol(df, time_freq)),
    "Maximum Drawdown (%)": (lambda df: drawdown(df).max().values * 100),
    "Underwater (%)": (lambda df: (drawdown(df) != 0).sum() / len(df) * 100),
    f"(%) Underwater >{20} %":
    (lambda df, treshold=0.20: (drawdown(df) > treshold).sum() / len(df) * 100)
}

## Exploratory Data Analysis (EDA) <a id='2.2'></a>

In [ ]:
g = sns.pairplot(df_factors, height=5, aspect=1.5, diag_kind="kde")

### Full sample period: 1927:2019

In [ ]:
Return_DD(df=df_factors, dic_labels=dic_fac, loc='upper center',time_frequency=FREQUENCY)
# plt.savefig('Cum_return_DD', bbox_inches='tight') # save plot

###  Period: 1927-1935
 - crash of 1930

In [ ]:
start, end = datetime.date(year=1927, month=1, day=1), datetime.date(year=1935,
                                                                     month=1,
                                                                     day=1)
Return_DD(df=df_factors,
          dic_labels=dic_fac,
          loc='upper left',
          start=start,
          end=end,
          time_frequency=FREQUENCY)

### Period: 1998-2010
 - chrash of 2000 with the internet buble
 - chrash of 2007

In [ ]:
start, end = datetime.date(year=1998, month=1, day=1), datetime.date(year=2010,
                                                                     month=1,
                                                                     day=1)
Return_DD(df=df_factors,
          dic_labels=dic_fac,
          loc='upper center',
          start=start,
          end=end,
          time_frequency=FREQUENCY)

In all serious drawdown periods, momentum crashes when the market starts rebouncing. This is in line what's described in the literature. Also note the negative correlation between value and momentum and momentum and the market.

In [ ]:
dic = {i: kappa_ratio(df_factors, order=i, time_frequency=FREQUENCY) for i in (np.arange(1, 6.5, 0.25))}
out = pd.DataFrame(dic).T
out = out.rename(columns={0: "Mkt-RF", 1: "HML", 2: "WML", 3: "HML-WML"})
fig = figsize = (15, 8)
fig, (ax1) = plt.subplots(1, 1, figsize=(15, 6))
for i, c in enumerate(out.columns):
    ax1.plot(out.index, out[c], label=str(c), linestyle="--")
    ax1.set_xlabel("$j$")
    ax1.set_ylabel("Kappa Ratio")
    ax1.legend(loc='best')

Increasing the power ($j$) of the `kappa ratio` gives higher weight to deviations below the treshold ($\tau$=0). It is interesting to see that the  momenetum strategy intially has a higher kappa ratio, but get surpassed around $j=3$ by the value portfolio. This means the momentum portfolio contains some large negative returns (tail events).

In [ ]:
round(perf_stat(df=df_factors, dic_perf=perf_stats_dic , stats="all"), 3)

In [ ]:
round(df_factors.describe(), 2)

**Some observations with respect to performance assessment of the four portfolios**

- Momentum seems to be the more "risky" strategy when looking at `skewness, kurtosis, kappa ratio` though it has not the highest `drawdown`. Note that the Sharpe ratio does not reveal this sort of information ( IMO, it does not make much sense to look at the Sharpe ratio when returns are not normally distributed).


- **Around 93%** of the time, the portfolios are in a state of drawdown (a state of regret). Meaning if we pick a random day, this is very high likelihood, there is some point in time we had more money. I think this an underappreciated fact and has a big impact on emotional state of investors. Note, this is not something specific for value or momentum investing. It is a feature of most investment strategies (even very good ones, see [**180 years of market drawdown [8]**](https://www.youtube.com/watch?v=27x632vOjXk&t=3349s)).


- Big differences between the four portfolios when looking at the `(%) underwater > 20 %` (I kind of arbitrarily chose 20 % to indicate a big drawdown). The difference is espcially big between the value and momentum strategy (43% vs 17%). Momentum experiences two very big drawdown period 1930-1960 and 2009-today.


- Combining value and momentum (HML-WML) improves risk adjusted performance on almost all statistics, though this relation is time dependent and seems to have disappeared over the last decade.


In [ ]:
# Returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    axs[i].scatter(df_factors.index,
                   df_factors.iloc[:, i].values,
                   label=j,
                   color=set2[i],
                   s=8)
    axs[i].set_xlabel("Date")
    axs[i].set_ylabel("Raw returns")
    axs[i].legend(loc="best")
plt.tight_layout()
plt.suptitle('Returns', y=1.05, fontsize=22, fontweight="bold")

# Squared returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    axs[i].scatter(df_factors.index,
                   df_factors.iloc[:, i].values**2,
                   label=j,
                   color=set2[i],
                   s=8)
    axs[i].set_xlabel("Date")
    axs[i].set_ylabel("Squared returns")
    axs[i].legend(loc="best")
plt.tight_layout()
plt.suptitle('Squared returns', y=1.05, fontsize=22, fontweight="bold")

# Partial autocorrelation: returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    plot_pacf(df_factors.iloc[:, i].values,
              ax=axs[i],
              label=j,
              lags=25,
              color=set2[i])
    axs[i].set_xlabel("Lags")
    axs[i].set_ylabel('$\\rho$')
    axs[i].set_title("")
    #axs[i].set_ylim(-0.5,0.5)
    handles, labels = axs[i].get_legend_handles_labels()
    handles = handles[:-len(handles) // 3][0::1]
    labels = labels[:-len(handles) // 3][0::1]
    axs[i].legend(handles=handles, labels=labels, loc='best')
plt.tight_layout()
plt.suptitle('Partial autocorrelation raw returns',
             y=1.05,
             fontsize=22,
             fontweight="bold")

# Partial autocorrelation: squared returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    plot_pacf(abs(df_factors.iloc[:, i]).values,
              ax=axs[i],
              label=j,
              lags=25,
              color=set2[i])
    axs[i].set_xlabel("Lags")
    axs[i].set_ylabel('$\\rho$')
    axs[i].set_title("")
    #axs[i].set_ylim(-0.5,0.5)
    handles, labels = axs[i].get_legend_handles_labels()
    handles = handles[:-len(handles) // 3][0::1]
    labels = labels[:-len(handles) // 3][0::1]
    axs[i].legend(handles=handles, labels=labels, loc='best')
plt.tight_layout()
plt.suptitle('Partial autocorrelation absolute returns',
             y=1.05,
             fontsize=22,
             fontweight="bold")

# Partial autocorrelation: squared returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    plot_pacf(df_factors.iloc[:, i].values**2,
              ax=axs[i],
              label=j,
              lags=25,
              color=set2[i])
    axs[i].set_xlabel("Lags")
    axs[i].set_ylabel('$\\rho$')
    axs[i].set_title("")
    #axs[i].set_ylim(-0.5,0.5)
    handles, labels = axs[i].get_legend_handles_labels()
    handles = handles[:-len(handles) // 3][0::1]
    labels = labels[:-len(handles) // 3][0::1]
    axs[i].legend(handles=handles, labels=labels, loc='best')
plt.tight_layout()
plt.suptitle('Partial autocorrelation squared returns',
             y=1.05,
             fontsize=22,
             fontweight="bold")

# QQplot: simple returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    sm.qqplot(df_factors.iloc[:, i].values,
              fit=True,
              line='45',
              label=j,
              ax=axs[i],
              color=set2[i])
    axs[i].legend(loc="best")
plt.tight_layout()
plt.suptitle('Simple returns', y=1.05, fontsize=22, fontweight="bold")

# QQplot: log returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    sm.qqplot(np.exp(1 + df_factors.iloc[:, i].values / 100),
              fit=True,
              line='45',
              label=j,
              ax=axs[i],
              color=set2[i])
    axs[i].legend(loc="best")
plt.tight_layout()
plt.suptitle('Log returns', y=1.05, fontsize=22, fontweight="bold");

**some observations (all strategies)**

 - Returns are hetroskedastic:  return volatility is high during the 1930s, and one can argue that this period is a different regime, unlikely to repeat (some papers exclude the period 1927-1963 because of this, I personally disagree with this). In contrast, 1941–1962 is a period of relatively low return volatility. Thereafter, we observe similar behavior as in the 1930's.
 - Serial correlations in the simple returns is rather low, but much higher serial correlations in the absolute and squared returns (I am quite surprised, this is promising for the second part of the thesis).
 - Return are not normally distributed (simple and log).

In [ ]:
# Cumulative squared returns
fig, axs = plt.subplots(2, 2, figsize=(15, 8))
axs = axs.ravel()
for i, j in enumerate(df_factors.columns):
    axs[i].plot(df_factors.index, (df_factors.iloc[:, i]**2).cumsum(),
                label=j,
                color=set2[i],
                lw=4)
    axs[i].legend(loc="best")
    axs[i].set_ylabel("CSR")
    axs[i].set_xlabel("Date")
plt.tight_layout()
plt.suptitle('Cumulative squared returns (CSR)',
             y=1.05,
             fontsize=22,
             fontweight="bold");

**Cumulative Squared Returns (CSR)**

The CSR is a “quick and dirty” approach to look at volatility regimes. The variance, $\sigma_r^2$, of return can be expressed by the following, where $R$  is the random variable representing return:

\begin{equation}
\sigma_{r}^{2}=\mathrm{E}\left[R^{2}\right]-\mathrm{E}[R]^{2}
\end{equation}

If we take the maximum likelihood estimate (MLE) for the sample variance, $\hat{s}_r^2$ , and multiply both sides by the sample size, $n$ , we have

\begin{equation}
n \hat{s}_{r}^{2}=\sum_{t=1}^{n} r(t)^{2}-\frac{\left(\sum_{t=1}^{n} r(t)\right)^{2}}{n}
\end{equation}

For most return time series the term to the left of the minus sign is much larger than the term on the right. This leads to the following approximation:

\begin{equation}
n \hat{s}_{r}^{2} \simeq \sum_{t=1}^{n} r(t)^{2}
\end{equation}

Thus, when plotting the cumulative square returns (CSR), the slope of the CSR line is approximately the variance of those returns. If the return switches from one stable volatility regime to another, then this shift shows as an abrupt change in slope of the CSR. This is a simple but powerful exploratory data analysis technique. The idea is to gain insights that give intuition about the data. Of course, you want to make sure that our insights make sense, but at this stage I am not super concerned with the formalities.


Looking at the CSR it does appear that a piecewise model yields a reasonably good description. There are extended periods where the slope is roughly constant, and these are separated by definite knits showing rapid transition from one regime to another.

**Next steps (difficult in python)**

My next step was to approximate the CSR with a piecewise linear model and look how these slopes change over time. The slope of the $i^{th}$ line segment ($\tau$)  would  the estimate of the constant variance. If we multiple each variance by $12$ or $252$ (depending if you use monthly or daily data) and then take the square root, then we have an annualized estimate of the standard deviation:

\begin{equation}
\hat{S}_{\text { annual }}\left(\tau_{i}\right)=\sqrt{252 \times \hat{s}_{\text { daily }}\left(\tau_{i}\right)^{2}}
\end{equation}


Doing the above in the **R programming language** seems a lot more practical (there are some packages which identify changing points). So far, I have experimented a bit with this, the problem is determining the cutpoints to fit the piecewise model (a tried with a simple decidision tree which gave actually nice results). If I have time left, I will take another look at this. Suggestions are also welcome.

<a id='Chapter3'></a>

# Chapter 3: Simulation analysis Part 1

It seems likely that the characteristics of long-horizon returns are of central interest to investors saving for distant payoffs. If so, it is likely that the characteristics of long-horizon returns affect asset pricing in ways missed by traditionel models. Evidence on distributions of long-horizon returns is a logical first
step in remedying this deficiency. I use simulations to study distributions of U.S. stock returns the distributions of annual, and 3-, 5-, 10-, 20-, and 30-year returns on the  Market portfolio (Mkt-RF), value portfolio (HML), a momentum portfolio (WML)  and a 50-50 value-momentum portfolio (HML-WML) of U.S. stocks for January 1927 to February 2019 (henceforth 1927–2019). Sample sizes shrink quickly as the return horizon increases. I use (bootstrap) simulations to fill in details of long horizon return distributions. 

**Portfolios**

- market (Mkt-RF)
- Value (HML)
- Momentum (WML)
- 50-50 Value-Momentum (HML-WML) 

**simulations methods**

   - sample returns from a normal distributions using the $\mu$ and $\sigma$ estimated from the data
   - Independent identical distributed (i.i.d.) bootstrap
   - circular bootstrap
   - stationairy bootstrap

**statistics of interest**

I focus on the following three statistics which I consider important elements of any investment strategy:
 
 - geometric return (annualized) 
 - the number of times we end up with a positive return (based on geometric return)
 - Maximum Drawdown
 
Regarding the maximum drawdown, I am concerned whether this makes sense, since this is an extrema statistic.

**Investment horizon**

- 1-3-5-10-20-30 years

Regarding **geometric return**, I expect it to converge toward a normal distributions (this is a rather simple and well behaved statistic) as the investment horizon increases. I'm most of all curious about how fast the variance will drop as you increase the investment horizon.

Regarding **maximum drawdown**, I honestly don't know what to expect (maybe some kind of right skewed distribution?). As you increase the investment horizon you will experience larger maximum drawdown by construction (your biggest loss is in the future). Does it even makes sense to look at such a statistic in a bootstrap setting?

### Helper functions

 - `boot:` iterates over the bootstrap paths and puts it into a dataframe
 - `plot_realizations:` plots $n$ (default 50) randomly chosen paths to get and idea of the simulated returns paths. This is visualized when calling the simulation method.
 - `plot_summary_stats:` visualizes the histograms from all the statistics when calling the bootstrap method (the default setting is not showing since this is a bit too much output)
 - `investment_horizons:` wraps around the functions from above and iterates over the simulation methods
 - `permutation_plot:` visualizes the permuted sample statistic(s) of interest. Note this will show the same histogram as those in `plot_summary_stats`
 - `append_stats:` appends the calculated statistic together in a pandas dataframe for each:
      - strategy
      - simulation method
      - investment horizon

 - `performance_matrix:` calculates the summary statistics of the permuated histograms
 - `describe_dist:` shows the summary statisitcs (calculated in `performance_matrix:`)  in a multi index dataframe


### Class

 - `Simulations:` a wrapper around the function `investment_horizon`. It contains 5 methods to perform the simulation:
      - sample from a normal distribution
      - iid boostrap
      - mbb
      - cbb 
      - sb
      
      
All information regarding the bootstrap is stored into a dictionary **store_output_dic**. This dictionary consist out of several sub dictionaries. You can think of it as a tree structure: 

 -  At the highest level you have the **investment portfolios**
 
       - each portfolio contains the chosen **simulation methods**
       
            - each portfolio on his turn contains a dictionary **Total sample length and Investment horizon** which contains the calculated sample statistic(s)  
     

### Parameters


In [ ]:
perf_stats_dic.keys()  # statisitcs to choose form

In [ ]:
sum_stats = ['Geometric Return (%)', 'Maximum Drawdown (%)']
store_output_dic = {}  # store all the output in a dictionary

## Market Factor (Mkt-RF) <a id='3.1'></a>

In [ ]:
market_simulation = Simulation(data=df_factors.loc[:, "Mkt-RF"],
                               n_paths=N_PATHS,
                               blocksize=BLOCKSIZE,
                               stats=sum_stats,
                               perf_functions=perf_stats_dic,
                               investment_horizon=IH,
                               frequency=FREQUENCY,
                               plotting=PLOT_INTERMEDIATE,
                               store_output=store_output_dic)

market_simulation.normal()
market_simulation.iid_bootstrap()
#market_simulation.mbb_bootstrap()
market_simulation.cbb_bootstrap()
market_simulation.sb_bootstrap()

In [ ]:
market_simulation.store_output.keys()

## Value (HML) <a id='3.2'></a>

In [ ]:
value_simulation = Simulation(data=df_factors.loc[:, "HML"],
                              n_paths=N_PATHS,
                              blocksize=BLOCKSIZE,
                              stats=sum_stats,
                              perf_functions=perf_stats_dic,
                              investment_horizon=IH,
                              frequency=FREQUENCY,
                              plotting=PLOT_INTERMEDIATE,
                              store_output=store_output_dic)

value_simulation.normal()
value_simulation.iid_bootstrap()
# value_simulation.mbb_bootstrap()
value_simulation.cbb_bootstrap()
value_simulation.sb_bootstrap()

In [ ]:
value_simulation.store_output.keys()

## Momentum (WML) <a id='3.3'></a>

In [ ]:
momentum_simulation = Simulation(data=df_factors.loc[:, "WML"],
                                 n_paths=N_PATHS,
                                 blocksize=BLOCKSIZE,
                                 stats=sum_stats,
                                 perf_functions=perf_stats_dic,
                                 investment_horizon=IH,
                                 frequency=FREQUENCY,
                                 plotting=PLOT_INTERMEDIATE,
                                 store_output=store_output_dic)

momentum_simulation.normal()
momentum_simulation.iid_bootstrap()
# momentum_simulation.mbb_bootstrap()
momentum_simulation.cbb_bootstrap()
momentum_simulation.sb_bootstrap()

In [ ]:
momentum_simulation.store_output.keys()

## 50-50 value-momentum <a id='3.4'></a>

In [ ]:
val_mom_simulation = Simulation(data=df_factors.loc[:, "HML-WML"],
                                n_paths=N_PATHS,
                                blocksize=BLOCKSIZE,
                                stats=sum_stats,
                                perf_functions=perf_stats_dic,
                                investment_horizon=IH,
                                frequency=FREQUENCY,
                                plotting=PLOT_INTERMEDIATE,
                                store_output=store_output_dic)

val_mom_simulation.normal()
val_mom_simulation.iid_bootstrap()
# val_mom_simulation.mbb_bootstrap()
val_mom_simulation.cbb_bootstrap()
val_mom_simulation.sb_bootstrap()

In [ ]:
val_mom_simulation.store_output.keys()

## summarize results <a id='3.5'></a>

I will start by making some figures in order to summarizes the information efficiently. Next, I do the same thing numerically.


**Statistics:**
  
  - geometric return 
  - maximum drawdown
  - number of positive returns (based on the geometric return)

**Portfolios:**

 - Mkt-RF
 - HML
 - WML
 - HML-WML
 
**Simulation methods**

- normal
- iid bootstrapping
- cbb
- sb

**Investment horizons (year)**
- 1, 3, 5, 10, 20, 30

###  Visually

#### 1. Geometric return

In [ ]:
permutation_plot(dic=val_mom_simulation.store_output,
                 investment_h=30,
                 statistic='Geometric Return (%)',
                 period="1927-2019")

#### 2. Maximum Drawdown

In [ ]:
permutation_plot(dic=val_mom_simulation.store_output,
                 investment_h=30,
                 statistic='Maximum Drawdown (%)',
                 period="1927-2019")

In [ ]:
statistic = "Geometric Return (%)"
df_stat = append_stats(stat=statistic, dic=val_mom_simulation.store_output)
df_stat.tail()

above_zero = df_stat.groupby(
    ["Strategy", "Investment Horizon (year)", "Simulation technique"],
    sort=False)[statistic].apply(lambda g: (g < 0).sum()).reset_index(
        name='count')
above_zero.head()

#### 3. Positive return for different investment horizon (based on geometric return)

In [ ]:
f, axes = plt.subplots(2, 3, figsize=(20, 14))
axes = axes.ravel()
i = 0
for h, h_df in above_zero.groupby("Investment Horizon (year)"):

    sns.barplot(y="Simulation technique",
                x="count",
                hue="Strategy",
                data=h_df,
                ax=axes[i])
    axes[i].get_legend().set_visible(False)  # remove legend
    axes[i].set_ylabel("Simulation method")
    axes[i].set_title(f'Investment Horizon: {h} year',
                      fontsize=18,
                      fontweight="bold",
                      y=1.02)
    axes[i].grid(True, which='major', linestyle='-.', color='0.8')
    i += 1
plt.suptitle(f"Number of paths with a negative\
 final value conditioned on investment horizon and factor\n ({N_PATHS :,}\
 bootstrap samples, blocksize = {BLOCKSIZE} days)",
             y=1.15,
             fontsize=25,
             fontweight="bold",
             style="italic")

# remove unnecessary labels
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[4].set_ylabel("")
axes[5].set_ylabel("")
axes[0].set_xlabel("")
axes[1].set_xlabel("")
axes[2].set_xlabel("")
# set legend
plt.tight_layout()
axes[1].legend(bbox_to_anchor=(1.175, 1.275), ncol=4)

# save
plt.savefig('Positive_Return', bbox_inches='tight')

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(20, 10))
axes = axes.ravel()
i = 0
x = np.array([1, 2, 3, 4, 5, 6])
for s1, d1 in above_zero.groupby(["Strategy"], sort=False):
    for s2, d2 in d1.groupby("Simulation technique", sort=False):
        axes[i].plot(x, d2["count"], label=s2, linestyle='--', marker='o')

        axes[i].set_title(f'Investment Strategy: {s1}',
                          fontsize=18,
                          fontweight="bold",
                          y=1.02)
        axes[i].grid(True, which='major', linestyle='-.', color='0.8')

    axes[i].set_xticklabels(["1", "1", "3", "5", "10", "20", "30"])
    axes[i].set_xlabel("Investment horizon (year)")
    i += 1
axes[0].set_ylabel("count")
axes[0].set_xlabel("")
axes[2].set_ylabel("count")
axes[1].set_xlabel("")

plt.tight_layout()
axes[1].legend(bbox_to_anchor=(.3, 1.35), ncol=4)
plt.suptitle(
    f"Number of paths with a negative final value condtioned on investment\
horizon and factor\n ({N_PATHS :,} bootstrap samples, blocksize = {BLOCKSIZE} days)",
    y=1.2,
    fontsize=25,
    fontweight="bold",
    style="italic");
# save
#plt.savefig('Positive_Return_line', bbox_inches='tight')

###  Numeric Summary Tables

### Mkt-RF
   #### 1. Geometric return

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="Mkt-RF",
                       statistic="Geometric Return (%)",
                       sim_meth=["Normal", "IB", "CBB", "SB"],
                       nr_neg=True), 2)

#### 2. Maximum Drawdown

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="Mkt-RF",
                       statistic="Maximum Drawdown (%)",
                       nr_neg=False), 2)

### HML
   #### 1. Geometric return

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="HML",
                       statistic="Geometric Return (%)",
                       nr_neg=True), 2)

#### 2. Maximum Drawdown

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="HML",
                       statistic="Maximum Drawdown (%)",
                       nr_neg=False), 2)

### WML
   #### 1. Geometric return

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="WML",
                       statistic="Geometric Return (%)",
                       nr_neg=True), 2)

#### 2. Maximum Drawdown

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="WML",
                       statistic="Maximum Drawdown (%)",
                       nr_neg=False), 2)

### HML-WML
   #### 1. Geometric return

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="HML-WML",
                       statistic="Geometric Return (%)",
                       nr_neg=True), 2)

#### 2. Maximum Drawdown

In [ ]:
round(
    performance_matrix(dic=val_mom_simulation.store_output,
                       strategy="HML-WML",
                       statistic="Maximum Drawdown (%)",
                       nr_neg=False), 2)

There is a clear distinction between the distributions depending on the simulation assumptions:

 - the normal and i.i.d. bootstrap are similar
 - and the circular block bootstrap and stationary bootstrap are more alike
 
IMO the circular and stationary bootstrap givs a far more  realistic picture about expectations, since they (try to) preserve the non-normality and serial correlations. One limitation of the circular block bootstrap is the fixed block size. Different block sizes emphasize different periods or lengths of autocorrelation (memory). At the extremes you can take a block size so small (i.i.d. bootstrap) that no serial correlation is captured, and at the other end you could take a block size so large that you end up sampling the original series. However, in my initial experimentation there are many reasonable block sizes that generate realistic price paths, that overall capture the behavior of the original series. I guess this is also why results of the circular block bootstrap are very close to those obtained from the stationary bootstrap. 


**Geometric Return** 

Perhaps most interesting are the **frequencies of negative returns (# neg. R) for different factors under a investment horizon ranging from 1 year to 30 years**.
When evaluating portfolio performance, professional investors often put much weight on three or five years of past returns. The message from the results above is that chance is a big player in outcomes for these horizons. In the simulations, the expected premiums are more or less equal to their sample averages. Negative simulated premiums are then just chance results of high return volatility. By definition the variance will shrink as you increase the investor horizon, the question remains how fast?

As an example, **For a five-year investment horizon**, the estimated probabilities that a **value strategy** delivers a negative premium on a purely chance basis are substantial, ranging between 20-26% depending on the simulation assumptions. The estimated frequencies of negative premiums are bigger at shorter investment horiozns, even though we know that, by construction, the expected premiums are positive and large. For longer investment horizons the negative premium becomes less likely but still substantial for 10-year periods and they are also nontrivial for 20-year periods. 

**Maximum Drawdown**

 
The main message when looking at drawdowns is two-fold. First, it shows that value of and momentum strategies like the market also suffers large drawdowns. Moreover, if we falsely view returns as being well approximated by independent normal distributions, you would severely underestimate the magnitude of the worst likely drawdowns. Second,  if we account for the serial correlation structures, the observed large drawdowns are not that surprising at all. 

<a id='Chapter4'></a>
# Chapter: 4 Simulation Analysis Part 2

## Recent poor performance of the Momentum and Value factor 

The figure below shows that value and momentum factor did not deliver much return
in the **last 15 years (01-01-2004 to 28-02-2019)**. Are these factors broken or just experiencing a period of bad luck? To answer this question, I examine the role of bad luck in explaining the recent poor performance of factors. I start with measuring the performance of each factor over the last 15 years
ending February 2019 and use simulations to assess the likelihood of observing this level of
performance, given how the factors performed before these recent years. I compare factor performance from **January 2004** through **February 2019** to performance from:
 - **July 1963 through December 2003**. 
 - **January 1927 through December 2003**
 
Similar as in Chapter 3, I then simulate returns under different assumptions to assess which features of factor returns, if any, can account for their poor recent performance. Some Papers **only include data starting from 1963** because they argue that the period before is not representative anymore due to higher volatility. Therefore, I will analyze both periods separately and look whether there are notable differences. **The number of simulations is set at 10,000. and the blocksize and mean blocksize are both 126 days**. Finally, the bootstrap samples where I calculate the statistic of interest on are based on an investment horizon of 15 years.


### Helper functions
 - `p_values:`calculates the p-values
 - `p_table:`appends the p-values for different statistics , investment horizons, 
    strategies and simulation methods in a table

### Performance over the last recent 15 years

In [ ]:
start = datetime.date(year=2004, month=1, day=1)
df_15 = df_factors.loc[start:]
Return_DD(df=df_15, dic_labels=dic_fac, time_frequency=FREQUENCY)

In [ ]:
print(round(df_15.describe(), 3))
obs_15 = round(perf_stat(df_15, stats="all", dic_perf=perf_stats_dic), 3)
obs_15

### Performance  before the last 15 years starting from 1963

In [ ]:
start, end = datetime.date(year=1963, month=1, day=1), datetime.date(year=2004,
                                                                     month=1,
                                                                     day=1)
df_before_15_63 = df_factors.loc[start:end]
Return_DD(df=df_before_15_63, dic_labels=dic_fac, time_frequency=FREQUENCY)

### Performance  before the last 15 years starting from 1927 (all data)

In [ ]:
start, end = datetime.date(year=1927, month=1, day=1), datetime.date(year=2004,
                                                                     month=1,
                                                                     day=1)
df_before_15_27 = df_factors.loc[start:end]
Return_DD(df=df_before_15_27, dic_labels=dic_fac,time_frequency=FREQUENCY)

In [ ]:
perf_stats_dic.keys()

In [ ]:
sum_stats = ['Geometric Return (%)', 'Maximum Drawdown (%)', 'Geometric SR']
investment_horizon = [15]  # in years
# store all the output in a dictionary
store_output_dic_63, store_output_dic_27 = {}, {}

## Simulation Period: 01-01-1963 till 31-12-2003 <a id='4.1'></a>
###  Market <a id='4.1.1'></a>

In [ ]:
market_simulation_15_63 = Simulation(data=df_before_15_63.loc[:, "Mkt-RF"],
                                     n_paths=N_PATHS,
                                     blocksize=BLOCKSIZE,
                                     stats=sum_stats,
                                     perf_functions=perf_stats_dic,
                                     investment_horizon=investment_horizon,
                                     frequency=FREQUENCY,
                                     plotting=PLOT_INTERMEDIATE,
                                     store_output=store_output_dic_63)

market_simulation_15_63.normal()
market_simulation_15_63.iid_bootstrap()
market_simulation_15_63.cbb_bootstrap()
market_simulation_15_63.sb_bootstrap()

In [ ]:
market_simulation_15_63.store_output.keys()

### Value (HML) <a id='4.1.2'></a>

In [ ]:
value_simulation_15_63 = Simulation(data=df_before_15_63.loc[:, "HML"],
                                     n_paths=N_PATHS,
                                     blocksize=BLOCKSIZE,
                                     stats=sum_stats,
                                     perf_functions=perf_stats_dic,
                                     investment_horizon=investment_horizon,
                                     frequency=FREQUENCY,
                                     plotting=PLOT_INTERMEDIATE,
                                     store_output=store_output_dic_63)

value_simulation_15_63.normal()
value_simulation_15_63.iid_bootstrap()
value_simulation_15_63.cbb_bootstrap()
value_simulation_15_63.sb_bootstrap()

In [ ]:
value_simulation_15_63.store_output.keys()

###  Momentum (WML) <a id='4.1.3'></a>

In [ ]:
momentum_simulation_15_63 = Simulation(data=df_before_15_63.loc[:, "WML"],
                                       n_paths=N_PATHS,
                                       blocksize=BLOCKSIZE,
                                       stats=sum_stats,
                                       perf_functions=perf_stats_dic,
                                       investment_horizon=investment_horizon,
                                       frequency=FREQUENCY,
                                       plotting=PLOT_INTERMEDIATE,
                                       store_output=store_output_dic_63)

momentum_simulation_15_63.normal()
momentum_simulation_15_63.iid_bootstrap()
momentum_simulation_15_63.cbb_bootstrap()
momentum_simulation_15_63.sb_bootstrap()

In [ ]:
momentum_simulation_15_63.store_output.keys()

### Value-Momentum (HML-WML) <a id='4.1.3'></a>

In [ ]:
val_mom_simulation_15_63 = Simulation(data=df_before_15_63.loc[:, "HML-WML"],
                                      n_paths=N_PATHS,
                                      blocksize=BLOCKSIZE,
                                      stats=sum_stats,
                                      perf_functions=perf_stats_dic,
                                      investment_horizon=investment_horizon,
                                      frequency=FREQUENCY,
                                      plotting=PLOT_INTERMEDIATE,
                                      store_output=store_output_dic_63)

val_mom_simulation_15_63.normal()
val_mom_simulation_15_63.iid_bootstrap()
val_mom_simulation_15_63.cbb_bootstrap()
val_mom_simulation_15_63.sb_bootstrap()

In [ ]:
val_mom_simulation_15_63.store_output.keys()

## Simulation Period: 01-01-1927 till 31-12-2003 <a id='4.2'></a>

### Market (Mkt-RF) <a id='4.2.1'></a>

In [ ]:
market_simulation_15_27 = Simulation(data=df_before_15_27.loc[:, "Mkt-RF"],
                                     n_paths=N_PATHS,
                                     blocksize=BLOCKSIZE,
                                     stats=sum_stats,
                                     perf_functions=perf_stats_dic,
                                     investment_horizon=investment_horizon,
                                     frequency=FREQUENCY,
                                     plotting=PLOT_INTERMEDIATE,
                                     store_output=store_output_dic_27)

market_simulation_15_27.normal()
market_simulation_15_27.iid_bootstrap()
market_simulation_15_27.cbb_bootstrap()
market_simulation_15_27.sb_bootstrap()

In [ ]:
market_simulation_15_27.store_output.keys()

###  Value (HML) <a id='4.2.2'></a>

In [ ]:
value_simulation_15_27 = Simulation(data=df_before_15_27.loc[:, "HML"],
                                     n_paths=N_PATHS,
                                     blocksize=BLOCKSIZE,
                                     stats=sum_stats,
                                     perf_functions=perf_stats_dic,
                                     investment_horizon=investment_horizon,
                                     frequency=FREQUENCY,
                                     plotting=PLOT_INTERMEDIATE,
                                     store_output=store_output_dic_27)

value_simulation_15_27.normal()
value_simulation_15_27.iid_bootstrap()
value_simulation_15_27.cbb_bootstrap()
value_simulation_15_27.sb_bootstrap()

In [ ]:
value_simulation_15_27.store_output.keys()

### Momentum (WML) <a id='4.2.3'></a>

In [ ]:
momentum_simulation_15_27 = Simulation(data=df_before_15_27.loc[:, "WML"],
                                     n_paths=N_PATHS,
                                     blocksize=BLOCKSIZE,
                                     stats=sum_stats,
                                     perf_functions=perf_stats_dic,
                                     investment_horizon=investment_horizon,
                                     frequency=FREQUENCY,  
                                     plotting=PLOT_INTERMEDIATE,
                                     store_output=store_output_dic_27)

momentum_simulation_15_27.normal()
momentum_simulation_15_27.iid_bootstrap()
momentum_simulation_15_27.cbb_bootstrap()
momentum_simulation_15_27.sb_bootstrap()

In [ ]:
momentum_simulation_15_27.store_output.keys()

### Value-Momentum (HML-WML) <a id='4.2.4'></a>

In [ ]:
val_mom_simulation_15_27 = Simulation(data=df_before_15_27.loc[:, "HML-WML"],
                                     n_paths=N_PATHS,
                                     blocksize=BLOCKSIZE,
                                     stats=sum_stats,
                                     perf_functions=perf_stats_dic,
                                     investment_horizon=investment_horizon,
                                      frequency=FREQUENCY,
                                     plotting=PLOT_INTERMEDIATE,
                                     store_output=store_output_dic_27)

val_mom_simulation_15_27.normal()
val_mom_simulation_15_27.iid_bootstrap()
val_mom_simulation_15_27.cbb_bootstrap()
val_mom_simulation_15_27.sb_bootstrap()

In [ ]:
val_mom_simulation_15_27.store_output.keys()

## Summarize results <a id='4.3'></a>

### Simulation Period: 01-01-1963 till 31-12-2003 <a id='4.3.1'></a>

In [ ]:
sum_stats

In [ ]:
statistic = 'Geometric Return (%)'  # could choose any of the statistic above
dic_output_63 = val_mom_simulation_15_63.store_output
df_stat_63 = append_stats(stat=statistic, dic=dic_output_63)
df_stat_63.head()

### Geometric Return

In [ ]:
# observed performance over the last 15 years
perf_stat(df=df_15, stats=["Geometric Return (%)"], dic_perf=perf_stats_dic)

In [ ]:
obs_T = perf_stat(df=df_15, stats=["Geometric Return (%)"],dic_perf=perf_stats_dic).values.tolist()[0]
permutation_plot(dic=val_mom_simulation_15_63.store_output,
                 investment_h=15,
                 statistic='Geometric Return (%)',
                 period="1963-2004",
                 obs_stats=obs_T)

### Maximum Drawdown

In [ ]:
# observed maximum dd over the last 15 years
perf_stat(df=df_15, stats=["Maximum Drawdown (%)"],dic_perf=perf_stats_dic)

In [ ]:
obs_T = perf_stat(df=df_15, stats=["Maximum Drawdown (%)"],dic_perf=perf_stats_dic).values.tolist()[0]
permutation_plot(dic=val_mom_simulation_15_63.store_output,
                 investment_h=15,
                 statistic='Maximum Drawdown (%)',
                 period="1963-2004",
                 obs_stats=obs_T)

### Numeric Summary Table: 01-01-1963 till 31-12-2003

The table below shows the number of returns paths with a more extreme statistic (more to the left except for drawdown this is right). The number of simulations was set at 10,000. 

In [ ]:
p_table(stat=sum_stats,
        dic=val_mom_simulation_15_63.store_output,
        observed=df_15, 
        perf_stats_dic=perf_stats_dic)

**Observations regarding the geometric return**

Below I describe my observations for the **annualized geometric return**, similar conclusions hold for other performance statistics (except maybe for maximum drawdown)

The permuted distributions excludes the 15-year period ending 28 February 2019 from the analysis in creating
the 10,000 simulated histories of factor returns. If an investor uses these data to calibrate
expectations about performance over the next 15 years, **how probable would the actual
observed geometric annual returns seem?**

As in chapter 3, I use a blocksize of 126 for the circular block bootstrap and the average blocksize for the stationary bootstrap is also set at 126 days. The p-value associated with the observed  returns are very low except for the market portfolio. Relative to the powerful returns from before 2004, the recent performance of the value, momentum and the value-momentum strategies over the past 15 years is unexpectedly low. There certainly are differences between the different simulation methods, though the main conclusion does not change.

### Simulation Period: 01-01-1927 till 31-12-2003 <a id='4.3.2'></a>

### Geometric return

In [ ]:
obs_T = perf_stat(df=df_15, stats=["Geometric Return (%)"],dic_perf=perf_stats_dic).values.tolist()[0]
permutation_plot(dic=val_mom_simulation_15_27.store_output,
                 investment_h=15,
                 statistic='Geometric Return (%)',
                 period="1927-2004",
                 obs_stats=obs_T)

### Maximum Drawdown

In [ ]:
obs_T = perf_stat(df=df_15, stats=["Maximum Drawdown (%)"],dic_perf=perf_stats_dic).values.tolist()[0]
permutation_plot(dic=val_mom_simulation_15_27.store_output,
                 investment_h=15,
                 statistic="Maximum Drawdown (%)",
                 period="1927-2004",
                 obs_stats=obs_T)

### Numeric Summary Table: 01-01-1927 till 31-12-2003

The table below shows the number of returns paths with a more extreme statistic (more to to left except for drawdown this is right). The number of simulations was set at 10,000. 

In [ ]:
p_table(stat=sum_stats,
        dic=val_mom_simulation_15_27.store_output,
        observed=df_15,
        perf_stats_dic=perf_stats_dic)

**Observations regarding the geometric return**

I repeat the excercise from above, but when constructing the permuted distribution I use all data starting from 01-01-1927 till 31-12-2003. All other parameters are kept the same as before. Looking at the p-values in the table above, the probability of observing the performance for various statisitcs over the last 15 years increases,  but stays rather low. This is certainly the case for the value-momentum strategy.

<a id='Chapter5'></a>
# Chapter 5: Conclusion and discussion


**Investment horizon matters**
 - The probabilities that premiums are negative on a purely chance basis are substantial for the 1-year, 3-year, 5-year and 10-year periods. Chance is big player in outcomes for these horizons.
 - Large differences noticeable conditioned on the simulation assumptions and investment strategy: still the main message does not change.
 - Deciding on the (mean) block size is not trivial, but there are many reasonable (mean) block sizes that generate realistic price paths, that overall capture the behavior of the original series.
 - Any study such as this suffers from survivor bias: the US stock market was one of the most successful investment choices an investor could have made when looking back. As a result, we are probably underestimating/overestimating depending on the statistic.
 
**Recent past performance of value, momentum and a 50-50 value-momentum strategies are unexpectedly low**

It is no secret that value and momentum strategies have recently fallen far short of investor expectations. Is this a case of the factors being broken or have they just been unlucky over the last 15 years? The answer is probably a combination of both. Recent factor performance has been uncharacteristically bad given pre-2004 performance. Value (HML), momentum (WML) and 50-50 value-momentum (HML-WML) pre-2004 returns were likely inflated due to data mining and selection bias, and their post 2004 returns were likely depressed by crowding  as the factors gained widespread adoption. In contrary, the performance of the market portfolio (Mkt-RF) over the last 15 years is in line with historical expectations.

<a id='Chapter6'></a>
# Chapter 6: Limitations

**Assumptions being made**


- all data over the whole sample is representative: contrary to other people I do think that data from 1927 is still useful. I believe  events like the crash in 1930 can happen again (actually the magnitude of 2007 was close to 1930).
  
- the the sampled blocks are iid
 
- stationarity
 
- there are clear volatility regimes: these have clear impact on our bootstrap results. What are the implications?
 
- drawdown is a question mark for me
 
- survivorship bias: the US stock market was one of the best investment choices an investor could have made. Therefore results in Chapter 3 are probably underestimated (in the negative sense) and are not generalizable to other countries. Furthermore, even in the US, very few investment firms outperform (on various statistics) the portfolios considered in this study (even before slippage and transactions costs).
 
- I do not account for slippage and transaction cost. Slippage is the difference between where the computer signaled the entry and exit for a trade and where actual clients with actual money, entered and exited the market using the computers signals. By definition you would never have had the same price as the historical reported price since you would have moved the price most likely against you. There are however clever ways to minimize this cost. I do not think this is a problem for Chapter 4 as long as you compare apples with apples.

- I assume an investor reinvest his money (no withdrawals), this may not be realistic in reality

- other important assumptions?

<a id='Chapter7'></a>
# Chapter 7: Questions


1. Some general comments on the analysis: 

      - thoughts, suggestions, thing I should pay attention to...


2. Normally when bootstrapping the distribution of the statistic(s) of interest you use all data you have. However, since I am interested in the distribution of the statistic for a specific investment horizon, I condition on the bootstrap length. Is this problematic and what are the consequences? 

3. Regarding maximum drawdown, can I bootstrap such a statistic? I read some arguments about being careful when bootstrapping extrema statistics  **(https://stats.stackexchange.com/questions/119748/using-bootstrap-to-obtain-sampling-distribution-of-1st-percentile), [[6]](http://www.stat.rutgers.edu/home/mxie/stat586/handout/Bootstrap1.pdf)**. For WML and WML-HML, you get a bivariate distribution. I assume this has to do with the volatility regimes?

4. These volatility regimes have a clear impact on our bootstrap results. What are the implications?
5. What are the assumptions being made regarding the block bootstrap (cbb and sb): 
  
    - for example we assume these blocks are iid 
    - how reasonable are these approaches considered for this problem and under what circumstances are they not appropriate? 


6. Regarding the stationary bootstrap: when is this bootstrap method more appropriate compared to the circular block bootstrap?
7. What important assumptions did I made, not mentioned
8. I also looked at the moving block bootstrap, but got very similar result to the circular and stationary block bootstrap. I would like to have a better understanding what assumptions I have made and which bootstraps methods are most appropriate in what circumstances. Regarding the block size, several rules have been suggested when choosing the block the block length. So far I have found several reasonable block sizes that seem to generate realistic price paths, that overall capture the behavior of the original series. Furthermore, the application of stationary bootstrap is less sensitive to the choice of block length compared to the circular block bootstrap.

<a id='Chapter8'></a>
# Chapter 8: References

## Papers
- **[1]  [Arnott, R. D., Harvey, C. R., Kalesnik, V., & Linnainmaa, J. T. (2019). Alice’s Adventures in Factorland: Three Blunders  That Plague Factor Investing. Available at SSRN 3331680.](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3331680)**
- **[2]  [Fama, E. F., & French, K. R. (2018). Long-horizon returns. The Review of Asset Pricing Studies, 8(2), 232-252.](https://academic.oup.com/raps/article-abstract/8/2/232/4810768)**
- **[3]  [Fama, E. F., & French, K. R. (2012). Size, value, and momentum in international stock returns. Journal of financial economics, 105(3), 457-472.](https://www.sciencedirect.com/science/article/pii/S0304405X12000931)**
- **[4]  [Daniel, K., & Moskowitz, T. J. (2016). Momentum crashes. Journal of Financial Economics, 122(2), 221-247.](https://www.sciencedirect.com/science/article/pii/S0304405X16301490)**
- **[5]  [Radovanov, B., & Marcikić, A. (2014). A comparison of four different block bootstrap methods. Croatian Operational Research Review, 5(2), 189-202.](https://scholar.google.be/scholar?hl=nl&as_sdt=0%2C5&q=Radovanov%2C+B.%2C+%26+Marciki%C4%87%2C+A.+%282014%29.+A+comparison+of+four+different+block+bootstrap+methods.+Croatian+Operational+Research+Review%2C+5%282%29%2C+189-202&btnG=)**

- **[6] [Singh, K., & Xie, M. (2008). Bootstrap: a statistical method. Unpublished manuscript, Rutgers University, USA. Retrieved from http://www. stat. rutgers. edu/home/mxie/RCPapers/bootstrap. pdf.](http://www.stat.rutgers.edu/home/mxie/stat586/handout/Bootstrap1.pdf)**
- **[7] [Efron, B. (1992). Bootstrap methods: another look at the jackknife. In Breakthroughs in statistics (pp. 569-593). Springer, New York, NY.](https://scholar.google.be/scholar?hl=nl&as_sdt=0%2C5&q=Efron%2C+B.+%281979%29.+Bootstrap+methods%3A+Another+look+at+jackknife.+Ann.+Stat.+7%2C+1-26&btnG=)**

## Blogs, websites, presentations ...

- **[8]  http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html**
- **[9]  http://keplerianfinance.com/2013/06/the-relevance-of-history/**
- **[10]  https://www.youtube.com/watch?v=27x632vOjXk&t=3349s**
- **[11]  https://www.youtube.com/watch?v=BtiqH_7d2es&t=28s**
- **[12] http://www.blackarbs.com/blog/synthetic-data-generation-part-1-block-bootstrapping**
- **[13] https://arch.readthedocs.io/en/latest/bootstrap/timeseries-bootstraps.html**
- **[14] https://pandas-datareader.readthedocs.io/en/latest/remote_data.html**
- **[15] http://www.ievbras.ru/ecostat/Kiril/R/Biblio_N/R_Eng/Chernick2011.pdf?fbclid=IwAR1f8UAc6s7FAGykclIB0usn0U9RGibNf6sB5ug2z9DEx9DoCK7Q-8nMVLk**
- **[16] http://www.quantdevel.com/public/CSP2015/Talk/BootstrappingTimeSeriesData.pdf?fbclid=IwAR3FiKvHjfrg9zGZkm8aa07vMT15PkpxAYFLqvR50uBdGIjOgM0AY0nsd2U**